# Gaussian Process Classification step 8

### Methods
The Gaussian Process Classifier (GPC) is a probabilistic, non-parametric machine learning method. It is particularly well-suited for classification tasks where the decision boundaries are complex and not easily defined by simple linear models. GPC models the data distribution using a Gaussian process, which allows it to capture non-linear relationships in the data.

We will assess a few likely kernels and see which performs best on a subset of the data. This is done in the context of computational efficiency, as GPCs can be computationally heavy, especially when being performed in many dimensions.


### Workflow
- Gaussian Process Classifier:

    We will create an instance of the Gaussian Process Classifier with an RBF kernel and train it on the preprocessed data.

    We will iterate through the kernels defined in step 7 and see which kernel performs best on our downscaled data. The best performing kernel will then be chosen.

- Model Evaluation:

    We will evaluate the performance of the GPC model by measuring its accuracy and other relevant metrics.

In the next notebook regarding steps, 9 and 10, we will upscale our model to more data using the preferred kernel chosen in this notebook.

### Install relevant dependencies

In [1]:
%%capture
!pip install cartopy
!pip install gpflow

In [8]:
import sys
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import resample
sys.path.append("/Users/Lisanne/Documents/AI4ER/PhD/GPs")

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import accuracy_score, classification_report

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Specify the file path for the pickle file
mean_df = pd.read_pickle("/content/drive/MyDrive/ai4er/GPs/data/processed/ml_ready/coldwarmclass.pkl")
#mean_df = pd.read_pickle("/content/drive/MyDrive/ai4er/GPs/data/processed/ml_ready/means_8.pkl")

## Data preparation: recap from 1-4

### Sample down dataframe
ITP and Argos data from 2006, 2007 and 2008 contain 269,671 datapoints.
For this interpolation task, the dimensionality D = 4, as we will use latitude, longitude, depth and time. We have N = 269671 collected datapoints.

To cut down on computational costs, the dataframe will be sampled down first. By grouping by latitude and longitude, the spatial variation will be maintained.

In [5]:
import pandas as pd
import numpy as np

# Group by latitude and longitude
grouped_df = mean_df.groupby(['latitude', 'longitude'])

# Define the target number of samples per group (adjust as needed)
target_samples = 5

# Initialize an empty DataFrame for the balanced samples
new_df = pd.DataFrame()

# Iterate over groups and randomly subsample to achieve balance
for group_name, group_data in grouped_df:
    if len(group_data) >= target_samples:
        # If the group has enough samples, randomly subsample
        sampled_data = group_data.sample(target_samples, replace=False)
    else:
        # If the group has fewer samples, use all of them
        sampled_data = group_data
    new_df = pd.concat([new_df, sampled_data])

# Reset the index of the balanced DataFrame
new_df.reset_index(drop=True, inplace=True)

In [6]:
merged_df = new_df

# Count the occurrences of each class in the "WaterMass" column
class_counts = merged_df["WaterMass"].value_counts()

# Find the count of the third class
desired_class_count = class_counts[3]

# Create an empty DataFrame to store the balanced data
balanced_df = pd.DataFrame()

# Iterate over unique classes and balance the samples based on the count of the third class
for unique_class in merged_df["WaterMass"].unique():
    class_data = merged_df[merged_df["WaterMass"] == unique_class]
    class_balanced = resample(class_data, n_samples=desired_class_count, random_state=42)
    balanced_df = pd.concat([balanced_df, class_balanced])

In [9]:
ref_latitude = 50
ref_longitude = -100

# Function to calculate angle between two points and the reference point
def calculate_angle(lat, lon):
  lat_rad, lon_rad = map(math.radians, [lat, lon])
  ref_lat_rad, ref_lon_rad = map(math.radians, [ref_latitude, ref_longitude])

  delta_lon = lon_rad - ref_lon_rad

  y = math.sin(delta_lon) * math.cos(lat_rad)
  x = math.cos(ref_lat_rad) * math.sin(lat_rad) - (math.sin(ref_lat_rad) * math.cos(lat_rad) * math.cos(delta_lon))

  angle_rad = math.atan2(y, x)
  angle_deg = math.degrees(angle_rad)
  return angle_deg

# Calculate angle for each point in the DataFrame
balanced_df['bearing'] = balanced_df.apply(lambda row: calculate_angle(row['latitude'], row['longitude']), axis=1)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Calculate distances from each point to the reference point
balanced_df['distance'] = haversine(ref_latitude, ref_longitude, balanced_df['latitude'], balanced_df['longitude'])

# Assuming 'balanced_df' already contains a 'month' column
balanced_df['seasonal_sin'] = np.sin(2 * np.pi * balanced_df['month'] / 12)

## Data preparation
Gaussian processes (GPs) are a flexible and non-parametric tool for regression and classification tasks. However, for large datasets, exact GPs can become computationally expensive because their time and space complexity scales cubically with the number of data points.

Labels are encoded in a one-hot fashion, so that if if C = 4 (classes), and y = 2, y is encoded as y = [0,1,0,0].

In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

n_clusters_per_region = 80

# Use KMeans for clustering within each region
kmeans_clusters = KMeans(n_clusters=n_clusters_per_region, n_init=n_clusters_per_region, random_state=7)
balanced_df['cluster'] = kmeans_clusters.fit_predict(balanced_df[['latitude', 'longitude']])

# Determine which clusters to assign to train, val, and test
cluster_indices = balanced_df['cluster'].unique()

# Assuming 'cluster' is the column containing cluster labels in balanced_df
clusters = balanced_df['cluster'].unique()

# Split clusters into training and validation sets
train_clusters, split_clusters = train_test_split(clusters, test_size=0.3, random_state=42)
val_clusters, test_clusters = train_test_split(split_clusters, test_size=0.4, random_state=42)

# Create training and validation datasets
train_dataset = balanced_df[balanced_df['cluster'].isin(train_clusters)]
val_dataset = balanced_df[balanced_df['cluster'].isin(val_clusters)]
test_dataset = balanced_df[balanced_df['cluster'].isin(test_clusters)]

# Verify the size of each dataset
print("Train Dataset Size:", len(train_dataset))
print("Validation Dataset Size:", len(val_dataset))
print("Test Dataset Size:", len(test_dataset))

Train Dataset Size: 1106
Validation Dataset Size: 249
Test Dataset Size: 293


In [15]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf

# List of feature names
feature_names = ['distance', 'bearing', 'depth', 'seasonal_sin']

# Extract features and target variable from train_dataset
X_train_dataset = train_dataset[feature_names]
y_train_dataset = train_dataset['WaterMass']

# Extract features and target variable from val_dataset
X_val_dataset = val_dataset[feature_names]
y_val_dataset = val_dataset['WaterMass']

# Extract features and target variable from test_dataset
X_test_dataset = test_dataset[feature_names]
y_test_dataset = test_dataset['WaterMass']

# Encode categorical labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded_dataset = label_encoder.fit_transform(y_train_dataset).reshape(-1, 1)
y_val_encoded_dataset = label_encoder.transform(y_val_dataset).reshape(-1, 1)
y_test_encoded_dataset = label_encoder.transform(y_test_dataset).reshape(-1, 1)

# Reshape without creating an additional dimension
y_train_encoded_dataset = y_train_encoded_dataset.reshape(-1)
y_val_encoded_dataset = y_val_encoded_dataset.reshape(-1)
y_test_encoded_dataset = y_test_encoded_dataset.reshape(-1)

# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled_dataset = scaler.fit_transform(X_train_dataset)
X_val_scaled_dataset = scaler.transform(X_val_dataset)
X_test_scaled_dataset = scaler.transform(X_test_dataset)

# GaussianProcessClassifier

This function, `evaluate_kernel`, assesses the performance of a given Gaussian process kernel using the `GaussianProcessClassifier` from scikit-learn. The function takes as input the training and validation features (`X_train` and `X_val`), corresponding labels (`y_train` and `y_val`), and the kernel to be evaluated.

### Parameters:

- `X_train`: Training features.
- `y_train`: Training labels.
- `X_val`: Validation features.
- `y_val`: Validation labels.
- `kernel`: The Gaussian process kernel to be evaluated.

### Returns:

- `accuracy`: Accuracy score of the model on the validation set.

### Functionality:

1. **GaussianProcessClassifier Initialization:**
    - Creates a `GaussianProcessClassifier` instance with the specified kernel and a set number of restarts for optimization (`n_restarts_optimizer`).

2. **Model Training:**
    - Fits the classifier to the training data (`X_train`, `y_train`).

3. **Prediction:**
    - Uses the trained model to predict labels for the validation set (`X_val`).

4. **Accuracy Calculation:**
    - Calculates the accuracy score by comparing predicted labels with true labels on the validation set.

5. **Prints Results:**
    - Displays the accuracy of the model with the current kernel.

6. **Confusion Matrix Visualization:**
    - Generates and visualizes the confusion matrix for the predictions on the validation set.


In [16]:
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, ExpSineSquared, ConstantKernel as C
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_kernel(X_train, y_train, X_val, y_val, kernel):
    """
    Evaluate a single kernel using GaussianProcessClassifier.

    Parameters:
    - X_train: Training features
    - y_train: Training labels
    - X_val: Validation features
    - y_val: Validation labels
    - kernel: Kernel to evaluate

    Returns:
    - accuracy: Accuracy for the given kernel
    """

    # Create GaussianProcessClassifier with the current kernel
    model_sklearn = GaussianProcessClassifier(kernel=kernel, n_restarts_optimizer=10)

    # Fit the model to the training data
    model_sklearn.fit(X_train, y_train)

    # Make predictions on the validation set
    predicted_classes = model_sklearn.predict(X_val)

    # Calculate accuracy on the validation set
    accuracy = accuracy_score(y_val, predicted_classes)

    # Print the accuracy for the current kernel
    print(f"Validation Accuracy ({str(kernel)}): {accuracy * 100:.2f}%")

    # Plot confusion matrix for the current kernel
    conf_matrix = confusion_matrix(y_val, predicted_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix ({str(kernel)})')
    plt.show()

    return accuracy, model_sklearn, predicted_classes

# Define kernels to evaluate on

## RBF Kernel for Classification

The Radial Basis Function (RBF) kernel, also known as the Gaussian kernel, is commonly used in Gaussian process-based classification tasks. Here's an explanation of why the RBF kernel is often preferred for classification:

##Smooth and Flexible Decision Boundaries:

- **Smoothness:** The RBF kernel generates smooth and continuous decision boundaries, making it suitable for capturing complex relationships between features. This smoothness is particularly advantageous in classification problems where decision boundaries might be intricate or nonlinear.

- **Flexibility:** The RBF kernel can model complex decision regions, allowing the classifier to adapt well to varying patterns in the data. This flexibility is crucial when dealing with datasets that exhibit intricate class structures.

##Versatility in Capturing Patterns:

- **Capturing Local Patterns:** The RBF kernel is effective at capturing local patterns and variations in the data. This is beneficial for classifying instances that may have different local characteristics within the feature space.

- **Implicit Feature Mapping:** The RBF kernel implicitly performs a non-linear feature mapping, allowing it to handle high-dimensional data and discover intricate relationships between features without explicitly expanding the feature space.

### Hyperparameter Control:

- **Length Scale Parameter:** The length scale parameter in the RBF kernel controls the smoothness and width of the decision boundaries. Adjusting this parameter provides a way to regulate the kernel's sensitivity to variations in the data, allowing fine-tuning of the model's generalization capacity.

### General Applicability:

- **Commonly Used:** The RBF kernel is a widely adopted choice in practice due to its effectiveness in capturing complex relationships and its suitability for a broad range of classification tasks.

In the provided code snippet, the RBF kernel is created with a specific length scale of 1.0. The choice of this length scale can be adjusted based on the characteristics of the dataset and the desired smoothness of the decision boundaries.


In [17]:
# Create an RBF kernel with the specified length scale
RBF_kernel = 1 * RBF(length_scale = 1.0)

In [ ]:
accuracy_rbf, model_rbf, predicted_rbf = evaluate_kernel(X_train_scaled_dataset, y_train_encoded_dataset,
                               X_val_scaled_dataset, y_val_encoded_dataset,
                               1.0 * RBF(length_scale=1.0))

In [ ]:
# Get the weights of the model
model_weights = model_rbf.kernel_
# Print or use the obtained information
print(f"Optimized Kernel Parameters: {model_weights}")

## Matern Kernel for Classification

The Matern kernel is another commonly used kernel in Gaussian process-based classification, offering certain advantages and characteristics. Let's explore why the 1.0 * Matern kernel might be chosen for classification tasks:

### Robustness and Flexibility:

- **Adaptive Smoothing:** The Matern kernel is a versatile choice that provides adaptive smoothing of decision boundaries. It offers a balance between the RBF kernel (infinitely differentiable) and the Laplace kernel (not differentiable), making it a robust option for diverse datasets.

- **Flexibility:** The Matern kernel is characterized by a parameter, often denoted as `nu`, which controls the smoothness of the decision boundaries. A higher value of `nu` results in a smoother function, while a lower value introduces more flexibility, allowing the kernel to capture intricate patterns in the data.

### Handles Noisy Data:

- **Noise Robustness:** The Matern kernel is known for its robustness to noisy data. In scenarios where the classification task involves data with inherent noise or uncertainties, the Matern kernel can provide stable and reliable predictions.

### Distance Metric Flexibility:

- **Distance Function Control:** The Matern kernel allows flexibility in choosing the distance metric for measuring dissimilarity between data points. This adaptability can be advantageous when dealing with datasets where different features may have distinct scales or importance.

### Hyperparameter Adjustments:

- **Length Scale Parameter:** Similar to the RBF kernel, the Matern kernel includes a length scale parameter (`length_scale`). Adjusting this parameter allows fine-tuning of the model's sensitivity to variations in the data, impacting the width and smoothness of decision boundaries.

### Effective for Spatially Varying Patterns:

- **Spatially Varying Patterns:** The Matern kernel is well-suited for capturing spatially varying patterns in the data. This is valuable in classification tasks where the relationships between features exhibit varying degrees of smoothness in different regions of the feature space.

Here, the 1.0 * Matern kernel is created with a specific length scale of 1.0. This length scale can be adjusted based on the characteristics of the dataset and the desired trade-off between smoothness and flexibility in the decision boundaries.


In [ ]:
matern_kernel = 1.0 * Matern(length_scale=1.0)

accuracy_matern = evaluate_kernel(X_train_scaled_dataset,
                                  y_train_encoded_dataset,
                                  X_val_scaled_dataset,
                                  y_val_encoded_dataset, matern_kernel)

## RBF Kernel with Different Length Scales

In the context of Gaussian process-based classification, the Radial Basis Function (RBF) kernel is often customized by varying its length scale for each input feature. This allows for an adaptive and feature-specific modeling of the relationships within the data. Let's explore the significance of using different length scales for the features `r`, `theta`, `depth`, and `seasonal_sin`:

### Feature-Specific Sensitivity:

- **`distance` (Distance from reference point):** The length scale for `distance` determines the range over which data points in radial distance influence each other. A larger length scale suggests that points farther apart contribute to the correlation, capturing patterns that extend over longer distances.

- **`theta` (Angular Position):** A smaller length scale for `theta` might be chosen to capture fine-grained patterns associated with small changes in angular position. It allows the kernel to be more sensitive to variations in the angular feature.

- **`depth` (Depth):** A length scale of 1.0 for `depth` indicates that data points within a depth range of 1.0 units influence each other. Adjusting the length scale can control how sensitive the kernel is to variations in depth.

- **`seasonal_sin` (Seasonal Sine Component):** A smaller length scale for `seasonal_sin` may be suitable for capturing rapid variations in the seasonal sine component. This allows the kernel to adapt to short-term fluctuations in the seasonal pattern.

### Trade-off between Smoothness and Sensitivity:

- **1.0:** Choosing a length scale of 1.0 generally balances the trade-off between smoothness and sensitivity. It provides a moderate level of sensitivity to variations in each feature without overly emphasizing local fluctuations.

- **0.1:** A smaller length scale of 0.1 increases sensitivity, making the kernel more responsive to local variations. This might be desirable when there are rapid changes or fine details in the data that need to be captured.

### Adaptability to Feature Characteristics:

- **Adaptation:** Adjusting the length scales allows the RBF kernel to adapt to the inherent characteristics of each feature. It provides a means to tailor the model to different scales and patterns present in the data.

In the provided code snippet, an RBF kernel is created with different length scales specified for `r`, `theta`, `depth`, and `seasonal_sin`. This customization enables the kernel to effectively capture diverse patterns and relationships within the multidimensional feature space.


In [ ]:
# Create an RBF kernel with the different length scales
length_scale = [1.0, 0.1, 1, 0.1]

lengthscale_kernel = RBF(length_scale=length_scale)

evaluate_kernel(X_train_scaled_dataset, y_train_encoded_dataset, X_val_scaled_dataset, y_val_encoded_dataset,
                lengthscale_kernel)